In [97]:
import pandas as pd
import numpy as np
import os 
%matplotlib inline

In [98]:
import sys
print(sys.path) 

['/home/hdd1/Projects/dizziness', '/home/skjang/anaconda3/envs/skorch/lib/python37.zip', '/home/skjang/anaconda3/envs/skorch/lib/python3.7', '/home/skjang/anaconda3/envs/skorch/lib/python3.7/lib-dynload', '', '/home/skjang/anaconda3/envs/skorch/lib/python3.7/site-packages', '/home/skjang/anaconda3/envs/skorch/lib/python3.7/site-packages/skorch-0.6.1.dev0-py3.7.egg', '/home/skjang/anaconda3/envs/skorch/lib/python3.7/site-packages/IPython/extensions', '/home/skjang/.ipython']


In [99]:
feature_withLab=False

In [100]:
savedir='/home/hdd1/Projects/dizziness/data'

In [101]:
# dizdata = pd.read_csv('/home/hdd1/DeepBrain Dropbox/Nunaps/SMART_Clinic/SK_Jang/19A_dizziness/data/dizziness_data_dimR4692col40_wohighmissing_190830.csv')
dizdata = pd.read_csv('/home/hdd1/DeepBrain Dropbox/Nunaps/SMART_Clinic/SK_Jang/19A_dizziness/data/dizziness_data_dimR4521col39_wohighmissing3000_191107.csv')
# dizdata = pd.read_csv('/home/hdd1/DeepBrain Dropbox/Nunaps/SMART_Clinic/SK_Jang/19A_dizziness/data/dizziness_data_dimR4521col42_wohighmissing4000_191107.csv')
contvar = pd.read_csv('/home/hdd1/DeepBrain Dropbox/Nunaps/SMART_Clinic/SK_Jang/19A_dizziness/data/dizziness_data_190820_continousVar.csv').iloc[:,0].to_list()

In [102]:
varlist = pd.read_csv('/home/hdd1/DeepBrain Dropbox/Nunaps/SMART_Clinic/SK_Jang/19A_dizziness/data/excluededVariables.csv')

In [103]:
rmvar=varlist.loc[varlist.exclude==1,:]['Feature Id']

In [104]:
dizdata.head()

,ID,arrival_date,Central.dizziness,Sex.M1F0,age,dizziness,Hypertension,Diabetes,smoking,chronic.kidney.disease,...,Glucose,Creatinine,Uric acid,Total cholesterol,Total protein,BUN,Sodium,Potassium,CRP,Revisit
0,21425372,2010-01-01 14:59:00,0,1,49,1,1,0,1,0,...,115.0,0.6,5.1,145.0,8.1,11.0,137.0,4.9,0.10,0
1,15963709,2010-01-02 09:42:00,0,0,60,0,1,0,0,0,...,185.0,0.7,4.6,182.0,6.9,14.0,137.0,3.8,0.10,0
2,23695696,2010-01-02 18:44:00,0,1,61,0,0,0,0,0,...,96.0,0.9,7.0,162.0,6.5,16.0,142.0,4.9,0.10,0
3,28910718,2010-01-03 09:11:00,0,0,58,0,1,1,0,0,...,130.0,0.6,3.0,151.0,6.5,12.0,137.0,4.3,0.10,0
4,39551342,2010-01-03 12:52:00,0,0,70,0,0,0,0,0,...,155.0,0.5,2.6,226.0,6.7,18.0,138.0,3.8,0.25,0


In [105]:
dizdata.shape 

(4521, 39)

In [106]:
len(dizdata.ID.unique())

4426

In [107]:
dizdata.ID.value_counts().value_counts()

1    4332
2      93
3       1
Name: ID, dtype: int64

In [108]:
contvar

['visit_id',
 'ID',
 'arrival_date',
 'age',
 '수축기혈압',
 '이완기혈압',
 '맥박',
 'WBC',
 'Hb',
 'Hct',
 'Platelet',
 'Prothrombin time (sec)',
 'Prothrombin time (percent)',
 'Prothrombin time (INR)',
 'aPTT',
 'Glucose',
 'Creatinine',
 'Uric acid',
 'Total cholesterol',
 'Total protein',
 'BUN',
 'Sodium',
 'Potassium',
 'Triglyceride',
 'HDL-cholesterol',
 'LDL-cholesterol',
 'CRP',
 'HbA1c',
 'Glucose,Stick']

In [109]:
img_var=[ 'MR,Diffusion (without enhance)',
 'MR,Acute stroke brain & Diffusion (with enhance)',
 'MR,Brain thrombolysis & Diffusion (with enhance)',
 'MR,Head angiography (with enhance)',
 'MR,Head & Neck angiography (with enhance)',
 'MR, Head Angio With & 3 Sequence',
 'CT,Brain(without enhance)',
 'CT,Brain (with enhance)',
 'CT,Head angiography (with enhance)']

In [110]:
total_var=dizdata.drop(columns=['ID','arrival_date','의식상태',
'Central.dizziness']).columns

In [111]:
total_var=total_var[~total_var.isin(rmvar)].tolist()

In [112]:
total_var

['Sex.M1F0',
 'age',
 'dizziness',
 'Hypertension',
 'Diabetes',
 'smoking',
 'chronic.kidney.disease',
 'coronary.artery.disease',
 'previous.stroke',
 'Hyperlipidemia',
 '수축기혈압',
 '이완기혈압',
 '맥박',
 'WBC',
 'Hct',
 'Platelet',
 'Glucose',
 'Creatinine',
 'Uric acid',
 'Total cholesterol',
 'BUN',
 'CRP',
 'Revisit']

In [113]:

if feature_withLab == True :
    feature_var = total_var 
else : 
    feature_var = ['Sex.M1F0',
     'age',
     'dizziness',
     'Hypertension',
     'Diabetes',
     'smoking',
     'coronary.artery.disease',
#      'chronic.kidney.disease',
     'previous.stroke',
     'Hyperlipidemia',
     '수축기혈압',
     '이완기혈압',
     '맥박',
     'Revisit'
                  ]


In [114]:
dizdata.shape

(4521, 39)

#### Exclude patients in Testset, which visited before 2012-10-13 12:28:00 

In [115]:
dizdata2=dizdata.copy()
dizdata2['Testset']=dizdata2.arrival_date>'2012-10-03 12:28:00'
trainID=np.unique(dizdata2.loc[~dizdata2['Testset'],:]['ID'])
tmp2=dizdata2.loc[(~dizdata2.ID.isin(trainID))|(~dizdata2.Testset),:] 
dizdata=dizdata2

In [116]:
dizdata['Central.dizziness'].value_counts()

0    4100
1     421
Name: Central.dizziness, dtype: int64

In [117]:
dizdata.loc[dizdata['Revisit']==1,:]['Central.dizziness'].value_counts()

0    80
1    15
Name: Central.dizziness, dtype: int64

In [118]:
dizdata.loc[dizdata['Revisit']==0,:]['Central.dizziness'].value_counts()

0    4020
1     406
Name: Central.dizziness, dtype: int64

In [119]:
CD_revisitpat=dizdata.loc[dizdata.ID.isin(dizdata.loc[(dizdata['Revisit']==1)&dizdata['Central.dizziness']==1,:]['ID']),:]['ID'].drop_duplicates()

In [120]:
import re 
def deID(id): 
    nid=re.sub('^.....', '***', str(id))
    return nid

In [121]:
tmp=dizdata.loc[dizdata.ID.isin(CD_revisitpat),:].sort_values(['ID','arrival_date'])
tmp['ID']=tmp['ID'].apply(deID) 
tmp.head()
tmp.to_csv('CD_revisitpat.csv',index=False)

## Split testset

In [122]:
adm=dizdata['arrival_date']

In [123]:
adm=pd.to_datetime(adm)

In [124]:
import pandas as pd
import numpy as np
from math import ceil

In [125]:
# def train_test_split_sorted(X, y, test_size, dates):
#     """Splits X and y into train and test sets, with test set separated by most recent dates.

#     Example:
#     --------
#     >>> from sklearn import datasets

#     # Fake dataset:
#     >>> gen_data = datasets.make_classification(n_samples=10000, n_features=5)
#     >>> dates = np.array(pd.date_range('2016-01-01', periods=10000, freq='5min'))
#     >>> np.random.shuffle(dates)
#     >>> df = pd.DataFrame(gen_data[0])
#     >>> df['date'] = dates
#     >>> df['target'] = gen_data[1]

#     # Separate:
#     >>> X_train, X_test, y_train, y_test = train_test_split_sorted(df.drop('target', axis=1), df['target'], 0.33, df['date'])

#     >>> print('Length train set: {}'.format(len(y_train)))
#     Length train set: 8000
#     >>> print('Length test set: {}'.format(len(y_test)))
#     Length test set: 2000
#     >>> print('Last date in train set: {}'.format(X_train['date'].max()))
#     Last date in train set: 2016-01-28 18:35:00
#     >>> print('First date in test set: {}'.format(X_test['date'].min()))
#     First date in test set: 2016-01-28 18:40:00
#     """

#     n_test = ceil(test_size * len(X))

#     sorted_index = [x for _, x in sorted(zip(np.array(dates), np.arange(0, len(dates))), key=lambda pair: pair[0])]
#     train_idx = sorted_index[:-n_test]
#     test_idx = sorted_index[-n_test:]

#     if isinstance(X, (pd.Series, pd.DataFrame)):
#         X_train = X.iloc[train_idx]
#         X_test = X.iloc[test_idx]
#     else:
#         X_train = X[train_idx]
#         X_test = X[test_idx]
#     if isinstance(y, (pd.Series, pd.DataFrame)):
#         y_train = y.iloc[train_idx]
#         y_test = y.iloc[test_idx]
#     else:
#         y_train = y[train_idx]
#         y_test = y[test_idx]

#     return X_train, X_test, y_train.astype(int), y_test.astype(int)

In [126]:
def train_test_split_rmdup(X, y, test_size, dates, patientID):

    n_test = ceil(test_size * len(X))

    sorted_index = [x for _, x in sorted(zip(np.array(dates), np.arange(0, len(dates))), key=lambda pair: pair[0])]
    train_idx = sorted_index[:-n_test]
    test_idx_tmp = sorted_index[-n_test:]
    print(f'No. records in testset before excluding duplicated patients : {len(test_idx_tmp)}')
    X_train = X.iloc[train_idx]
    X_test_tmp = X.iloc[test_idx_tmp]
    
    # remove dupilicated patients in a testset
    dup_bool = patientID[test_idx_tmp].isin(patientID[train_idx]) #1357
    dup_ID = patientID[test_idx_tmp][dup_bool] #40
    test_idx = dup_bool[~dup_bool].index.tolist() #1350-40 = 1317
    print(f'No. Duplicated records : {sum(dup_bool)}') # 40
    print(f'No. Duplicated patients (unique) : {len(dup_ID.unique())}')
    print(f'No. patients (unique) in testset : {len(patientID[test_idx].unique())}')
    print(f'No. patients (unique) in trainset : {len(patientID[train_idx].unique())}')
    print(f'No. records in testset : {len(test_idx)}')
    X_test = X_test_tmp.loc[test_idx,:]
    
    if isinstance(y, (pd.Series, pd.DataFrame)):
        y_train = y.iloc[train_idx]
        y_test = y.iloc[test_idx]
    else:
        y_train = y[train_idx]
        y_test = y[test_idx]

    return X_train, X_test, y_train.astype(int), y_test.astype(int),dup_ID

In [127]:
X_train, X_test, y_train, y_test, dup_ID=train_test_split_rmdup(dizdata.loc[:,dizdata.columns.isin(feature_var)],dizdata['Central.dizziness'],0.3,adm,dizdata.ID)

No. records in testset before excluding duplicated patients : 1357
No. Duplicated records : 40
No. Duplicated patients (unique) : 40
No. patients (unique) in testset : 1310
No. patients (unique) in trainset : 3116
No. records in testset : 1317


In [128]:
import datetime
today_datever=datetime.datetime.now().strftime("%y%m%d")
# if today_datever=='191107':
#     today_datever='191107rmdup'

In [129]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
objs = [
    X_train, X_test, y_train, y_test
]
tmp='%s/Dizzdataset_noimputation_withLAB%s_%s.pkl' %(savedir,feature_withLab,today_datever)
print(tmp)
save_object(objs, tmp)

/home/hdd1/Projects/dizziness/data/Dizzdataset_noimputation_withLABFalse_200422.pkl


In [130]:
dizdata2.to_csv('%s/DizzinessDataset_withTestsetLabel_%s.csv' %(savedir,today_datever),index=False)

In [131]:
len(X_train)+len(X_test)

4481

In [132]:
dizdata2.head()

,ID,arrival_date,Central.dizziness,Sex.M1F0,age,dizziness,Hypertension,Diabetes,smoking,chronic.kidney.disease,...,Creatinine,Uric acid,Total cholesterol,Total protein,BUN,Sodium,Potassium,CRP,Revisit,Testset
0,21425372,2010-01-01 14:59:00,0,1,49,1,1,0,1,0,...,0.6,5.1,145.0,8.1,11.0,137.0,4.9,0.10,0,False
1,15963709,2010-01-02 09:42:00,0,0,60,0,1,0,0,0,...,0.7,4.6,182.0,6.9,14.0,137.0,3.8,0.10,0,False
2,23695696,2010-01-02 18:44:00,0,1,61,0,0,0,0,0,...,0.9,7.0,162.0,6.5,16.0,142.0,4.9,0.10,0,False
3,28910718,2010-01-03 09:11:00,0,0,58,0,1,1,0,0,...,0.6,3.0,151.0,6.5,12.0,137.0,4.3,0.10,0,False
4,39551342,2010-01-03 12:52:00,0,0,70,0,0,0,0,0,...,0.5,2.6,226.0,6.7,18.0,138.0,3.8,0.25,0,False


## Imputation

In [133]:
X_train.head()

,Sex.M1F0,age,dizziness,Hypertension,Diabetes,smoking,coronary.artery.disease,previous.stroke,Hyperlipidemia,수축기혈압,이완기혈압,맥박,Revisit
0,1,49,1,1,0,1,0,0,0,152,94,77,0
1,0,60,0,1,0,0,0,0,0,169,105,71,0
2,1,61,0,0,0,0,1,0,0,138,89,67,0
3,0,58,0,1,1,0,0,1,0,188,109,71,0
4,0,70,0,0,0,0,0,0,1,131,92,71,0


In [134]:
temp=X_train.iloc[0,:]

In [135]:
temp

Sex.M1F0                     1
age                         49
dizziness                    1
Hypertension                 1
Diabetes                     0
smoking                      1
coronary.artery.disease      0
previous.stroke              0
Hyperlipidemia               0
수축기혈압                      152
이완기혈압                       94
맥박                          77
Revisit                      0
Name: 0, dtype: int64

In [136]:
freqval=X_train.apply(lambda x : x.value_counts().sort_values(ascending=False).iloc[[0]].index) #DataFrame

In [137]:
contvar=X_train.columns[X_train.columns.isin(contvar)].to_list()

In [138]:
categval=X_train.loc[:,~X_train.columns.isin(contvar)].columns.to_list()

# if ~feature_withLab  :
#     categval=['chronic.kidney.disease' if x=='간질환' else x for x in categval]


In [139]:
freqval

,Sex.M1F0,age,dizziness,Hypertension,Diabetes,smoking,coronary.artery.disease,previous.stroke,Hyperlipidemia,수축기혈압,이완기혈압,맥박,Revisit
0,0,63,1,0,0,0,0,0,0,135,78,78,0


In [140]:
temp[categval]=freqval[categval].values.tolist()[0]

In [141]:
temp[contvar]=X_train[contvar].median() 

In [142]:
temp

Sex.M1F0                     0.0
age                         62.0
dizziness                    1.0
Hypertension                 0.0
Diabetes                     0.0
smoking                      0.0
coronary.artery.disease      0.0
previous.stroke              0.0
Hyperlipidemia               0.0
수축기혈압                      144.0
이완기혈압                       87.0
맥박                          73.0
Revisit                      0.0
Name: 0, dtype: float64

In [143]:
# fillval=pd.DataFrame(medval.copy())
# fillval.iloc[fillval.index.isin(categval),:]=catgfreqval.transpose()
fillval_dic=dict(zip(temp.index,temp))

In [144]:
len(fillval_dic.items())

13

In [145]:
# Imputate train/testset
X_train=X_train.fillna(value=fillval_dic)
X_test=X_test.fillna(value=fillval_dic)

In [146]:
# import pickle
# def save_object(obj, filename):
#     with open(filename, 'wb') as output:  # Overwrites any existing file.
#         pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)
        
# objs = [
#     X_train, X_test, y_train, y_test
# ]
# save_object(objs, '%s/Dizzdataset_simpleimputation_withLAB%s_%s.pkl' %(savedir,feature_withLab,today_datever))

In [147]:
print('%s/Dizzdataset_simpleimputation_withLAB%s_%s.pkl' %(savedir,feature_withLab,today_datever))

/home/hdd1/Projects/dizziness/data/Dizzdataset_simpleimputation_withLABFalse_200422.pkl


In [148]:
# X_train.to_csv('%s/X_train_simpleimputation_withLAB%s_%s.csv' %(savedir,feature_withLab,today_datever),index=False)
# X_test.to_csv('%s/X_test_simpleimputation_withLAB%s_%s.csv' %(savedir,feature_withLab,today_datever),index=False)
# y_train.to_csv('%s/y_train_simpleimputation_withLAB%s_%s.csv' %(savedir,feature_withLab,today_datever),index=False)
# y_test.to_csv('%s/y_test_simpleimputation_withLAB%s_%s.csv' %(savedir,feature_withLab,today_datever),index=False)

In [149]:
freqval

,Sex.M1F0,age,dizziness,Hypertension,Diabetes,smoking,coronary.artery.disease,previous.stroke,Hyperlipidemia,수축기혈압,이완기혈압,맥박,Revisit
0,0,63,1,0,0,0,0,0,0,135,78,78,0


In [150]:
seldata=dizdata.copy()

In [151]:
seldata=seldata.loc[~(seldata.ID.isin(dup_ID)&seldata.Testset),:]

In [152]:
dizdata.shape

(4521, 40)

In [153]:
X_train.shape

(3164, 13)

In [154]:
print(100*sum(y_train==1)/len(y_train))
print(100*sum(y_test==1)/len(y_test))
print(100*sum(y_train==0)/len(y_train))

9.102402022756005
9.567198177676538
90.89759797724399


In [155]:
# seldata['Train']=seldata.index.isin(X_train.index)

In [156]:
wl=False

In [157]:
len(seldata)

4481

In [158]:
seldata['Sex.M1F0'].value_counts()

0    2644
1    1837
Name: Sex.M1F0, dtype: int64

In [159]:
seldata['Sex.M1F0'].value_counts()/len(seldata)

0    0.590047
1    0.409953
Name: Sex.M1F0, dtype: float64

In [160]:
seldata['age'].mean()

61.72907833072975

In [161]:
seldata['age'].std()

11.620420575488177

In [162]:
uniquepat=seldata.loc[~seldata['ID'].duplicated(),:]

In [163]:
uniquepat.age.mean()

61.672616357885225

In [164]:
uniquepat.age.std()

11.619218650885232

In [165]:
uniquepat['Sex.M1F0'].value_counts()

0    2614
1    1812
Name: Sex.M1F0, dtype: int64

In [166]:
if ~feature_withLab  :
    categval=['chronic.kidney.disease' if x=='간질환' else x for x in categval]

In [167]:
categval

['Sex.M1F0',
 'dizziness',
 'Hypertension',
 'Diabetes',
 'smoking',
 'coronary.artery.disease',
 'previous.stroke',
 'Hyperlipidemia',
 'Revisit']

In [168]:
seldata.columns

Index(['ID', 'arrival_date', 'Central.dizziness', 'Sex.M1F0', 'age',
       'dizziness', 'Hypertension', 'Diabetes', 'smoking',
       'chronic.kidney.disease', 'coronary.artery.disease', 'previous.stroke',
       'Hyperlipidemia', '의식상태', '수축기혈압', '이완기혈압', '맥박', '간질환', '호흡기', '수술력',
       'Neoplasm', 'WBC', 'Hb', 'Hct', 'Platelet', 'Prothrombin time (sec)',
       'Prothrombin time (percent)', 'Prothrombin time (INR)', 'aPTT',
       'Glucose', 'Creatinine', 'Uric acid', 'Total cholesterol',
       'Total protein', 'BUN', 'Sodium', 'Potassium', 'CRP', 'Revisit',
       'Testset'],
      dtype='object')

In [169]:
feature_var

['Sex.M1F0',
 'age',
 'dizziness',
 'Hypertension',
 'Diabetes',
 'smoking',
 'coronary.artery.disease',
 'previous.stroke',
 'Hyperlipidemia',
 '수축기혈압',
 '이완기혈압',
 '맥박',
 'Revisit']

In [170]:
from tableone import TableOne
tab=TableOne(data=seldata, columns=feature_var, categorical=categval ,groupby='Central.dizziness', pval=True,isnull=True)

In [171]:
tab

Grouped by Central.dizziness                                                       
                                                    isnull             0             1    pval              ptest
variable                level                                                                                    
n                                                                   4067           414                           
Sex.M1F0                0                                0   2475 (60.9)    169 (40.8)  <0.001        Chi-squared
                        1                                    1592 (39.1)    245 (59.2)                           
age                                                      0   61.3 (11.5)   66.1 (11.8)  <0.001  Two Sample T-test
dizziness               0                                0   1856 (45.6)     87 (21.0)  <0.001        Chi-squared
                        1                                    2211 (54.4)    327 (79.0)                           
Hypertension            0                                0   2536 (62.4)    216 (52.2)  <0.001        Chi-squared
                        1                                    1531 (37.6)    198 (47.8)                           
Diabetes                0                                0   3536 (86.9)    314 (75.8)  <0.001        Chi-squared
                        1                                     531 (13.1)    100 (24.2)                           
smoking                 0                                0   3802 (93.5)    373 (90.1)   0.012        Chi-squared
                        1                                      265 (6.5)      41 (9.9)                           
coronary.artery.disease 0                                0   3572 (87.8)    337 (81.4)  <0.001        Chi-squared
                        1                                     495 (12.2)     77 (18.6)                           
previous.stroke         0                                0   3754 (92.3)    314 (75.8)  <0.001        Chi-squared
                        1                                      313 (7.7)    100 (24.2)                           
Hyperlipidemia          0                                0   2699 (66.4)    281 (67.9)   0.571        Chi-squared
                        1                                    1368 (33.6)    133 (32.1)                           
수축기혈압                                                    0  145.7 (22.0)  148.3 (23.6)   0.032  Two Sample T-test
이완기혈압                                                    0   88.1 (14.0)   87.7 (15.8)   0.584  Two Sample T-test
맥박                                                       0   74.6 (14.0)   75.2 (15.1)   0.425  Two Sample T-test
Revisit                 0                                0   4020 (98.8)    406 (98.1)   0.257        Chi-squared
                        1                                       47 (1.2)       8 (1.9)                           
[1] Warning, Hartigan's Dip Test reports possible multimodal distributions for: age, 맥박, 수축기혈압, 이완기혈압.
[2] Warning, Tukey test indicates far outliers in: 맥박, 이완기혈압.
[3] Warning, test for normality reports non-normal distributions for: age, 맥박, 수축기혈압, 이완기혈압.

In [172]:
tab.to_csv(f'{savedir}/../results/tableone/tableone_total_withLAB{wl}_{today_datever}.csv')
tab12=pd.read_csv(f'{savedir}/../results/tableone/tableone_total_withLAB{wl}_{today_datever}.csv')

In [196]:
tab12.tail()

,Unnamed: 0,Unnamed: 1,Grouped by Central.dizziness,Grouped by Central.dizziness.1,Grouped by Central.dizziness.2,Grouped by Central.dizziness.3,Grouped by Central.dizziness.4
19,Hyperlipidemia,,0,1368 (33.6),133 (32.1),0.571,Chi-squared
20,수축기혈압,NaN,0,145.7 ±22.0,148.3 ±23.6,0.032,Two Sample T-test
21,이완기혈압,NaN,0,88.1 ±14.0,87.7 ±15.8,0.584,Two Sample T-test
22,맥박,NaN,0,74.6 ±14.0,75.2 ±15.1,0.425,Two Sample T-test
24,Revisit,,0,47 (1.2),8 (1.9),0.257,Chi-squared


In [174]:
idx0=np.where((tab12.iloc[:,1]=='0.0')|(tab12.iloc[:,1]=='0'))[0]
v=tab12.loc[(idx0).tolist(),'Grouped by Central.dizziness'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Central.dizziness']=v
v=tab12.loc[(idx0).tolist(),'Grouped by Central.dizziness.3'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Central.dizziness.3']=v
v=tab12.loc[(idx0).tolist(),'Grouped by Central.dizziness.4'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Central.dizziness.4']=v
tab12=tab12[~tab12.index.isin(idx0)]
tab12['Unnamed: 1']=tab12['Unnamed: 1'].replace('1.0','')
tab12['Unnamed: 1']=tab12['Unnamed: 1'].replace('1','')

contidx=tab12['Grouped by Central.dizziness.4']=='Two Sample T-test'
for cn in ['Grouped by Central.dizziness.1','Grouped by Central.dizziness.2']:
    tab12.loc[contidx,cn]=tab12.loc[contidx,cn].replace('\(',u'\u00b1',regex=True)
    tab12.loc[contidx,cn]=tab12.loc[contidx,cn].replace('\)','',regex=True)
tab12=tab12.loc[(~(tab12['Unnamed: 0']=='variable')),]
nd=tab12.iloc[1,3:5].astype(int).sum()
tab12.to_excel(f'{savedir}/../results/tableone/tableone_target_N{nd}_{today_datever}.xlsx',index=False)

In [197]:
savedir

'/home/hdd1/Projects/dizziness/data'

In [175]:
wl=False

In [176]:
tab=TableOne(data=seldata, columns=feature_var+['Central.dizziness'], categorical=categval +['Central.dizziness'],groupby='Testset', pval=True,isnull=True)

In [177]:
tab

Grouped by Testset                                                       
                                          isnull         False          True    pval              ptest
variable                level                                                                          
n                                                         3164          1317                           
Sex.M1F0                0                      0   1873 (59.2)    771 (58.5)   0.709        Chi-squared
                        1                          1291 (40.8)    546 (41.5)                           
age                                            0   61.6 (11.5)   61.9 (11.8)   0.417  Two Sample T-test
dizziness               0                      0   1523 (48.1)    420 (31.9)  <0.001        Chi-squared
                        1                          1641 (51.9)    897 (68.1)                           
Hypertension            0                      0   1906 (60.2)    846 (64.2)   0.014        Chi-squared
                        1                          1258 (39.8)    471 (35.8)                           
Diabetes                0                      0   2738 (86.5)   1112 (84.4)   0.073        Chi-squared
                        1                           426 (13.5)    205 (15.6)                           
smoking                 0                      0   2946 (93.1)   1229 (93.3)   0.852        Chi-squared
                        1                            218 (6.9)      88 (6.7)                           
coronary.artery.disease 0                      0   2747 (86.8)   1162 (88.2)   0.215        Chi-squared
                        1                           417 (13.2)    155 (11.8)                           
previous.stroke         0                      0   2876 (90.9)   1192 (90.5)   0.724        Chi-squared
                        1                            288 (9.1)     125 (9.5)                           
Hyperlipidemia          0                      0   2102 (66.4)    878 (66.7)   0.908        Chi-squared
                        1                          1062 (33.6)    439 (33.3)                           
수축기혈압                                          0  145.5 (22.0)  147.0 (22.7)   0.035  Two Sample T-test
이완기혈압                                          0   87.9 (14.1)   88.5 (14.3)   0.149  Two Sample T-test
맥박                                             0   74.5 (14.1)   75.0 (14.3)   0.297  Two Sample T-test
Revisit                 0                      0   3116 (98.5)   1310 (99.5)   0.010        Chi-squared
                        1                             48 (1.5)       7 (0.5)                           
Central.dizziness       0                      0   2876 (90.9)   1191 (90.4)   0.665        Chi-squared
                        1                            288 (9.1)     126 (9.6)                           
[1] Warning, Hartigan's Dip Test reports possible multimodal distributions for: age, 맥박, 수축기혈압, 이완기혈압.
[2] Warning, Tukey test indicates far outliers in: 맥박, 수축기혈압, 이완기혈압.
[3] Warning, test for normality reports non-normal distributions for: age, 맥박, 수축기혈압, 이완기혈압.

In [178]:
tab.to_csv(f'{savedir}/../results/tableone/tableone_traintest_withLAB{wl}_{today_datever}.csv')
tab12=pd.read_csv(f'{savedir}/../results/tableone/tableone_traintest_withLAB{wl}_{today_datever}.csv')

In [179]:
tab12.head()

,Unnamed: 0,Unnamed: 1,Grouped by Testset,Grouped by Testset.1,Grouped by Testset.2,Grouped by Testset.3,Grouped by Testset.4
0,NaN,NaN,isnull,False,True,pval,ptest
1,variable,level,NaN,NaN,NaN,NaN,NaN
2,n,NaN,NaN,3164,1317,NaN,NaN
3,Sex.M1F0,0,0,1873 (59.2),771 (58.5),0.709,Chi-squared
4,Sex.M1F0,1,NaN,1291 (40.8),546 (41.5),NaN,NaN


In [180]:
idx0=np.where((tab12.iloc[:,1]=='0.0')|(tab12.iloc[:,1]=='0'))[0]
v=tab12.loc[(idx0).tolist(),'Grouped by Testset'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Testset']=v
v=tab12.loc[(idx0).tolist(),'Grouped by Testset.3'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Testset.3']=v
v=tab12.loc[(idx0).tolist(),'Grouped by Testset.4'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Testset.4']=v
tab12=tab12[~tab12.index.isin(idx0)]
tab12['Unnamed: 1']=tab12['Unnamed: 1'].replace('1.0','')
tab12['Unnamed: 1']=tab12['Unnamed: 1'].replace('1','')

contidx=tab12['Grouped by Testset.4']=='Two Sample T-test'
for cn in ['Grouped by Testset.1','Grouped by Testset.2']:
    tab12.loc[contidx,cn]=tab12.loc[contidx,cn].replace('\(',u'\u00b1',regex=True)
    tab12.loc[contidx,cn]=tab12.loc[contidx,cn].replace('\)','',regex=True)
tab12=tab12.loc[(~(tab12['Unnamed: 0']=='variable')),]
nd=tab12.iloc[1,3:5].astype(int).sum()
tab12.to_excel(f'{savedir}/../results/tableone/tableone_traintest_N{nd}_{today_datever}.xlsx',index=False)

In [195]:
feature_var

['Sex.M1F0',
 'age',
 'dizziness',
 'Hypertension',
 'Diabetes',
 'smoking',
 'coronary.artery.disease',
 'previous.stroke',
 'Hyperlipidemia',
 '수축기혈압',
 '이완기혈압',
 '맥박',
 'Revisit']

In [191]:
tab2=TableOne(data=seldata.loc[seldata.Testset,:], columns=feature_var+['Central.dizziness'], categorical=categval ,groupby='Central.dizziness', pval=True,isnull=True)

In [192]:
tab2

Grouped by Central.dizziness                                                       
                                                    isnull             0             1    pval              ptest
variable                level                                                                                    
n                                                                   4067           414                           
Sex.M1F0                0                                0   2475 (60.9)    169 (40.8)  <0.001        Chi-squared
                        1                                    1592 (39.1)    245 (59.2)                           
age                                                      0   61.3 (11.5)   66.1 (11.8)  <0.001  Two Sample T-test
dizziness               0                                0   1856 (45.6)     87 (21.0)  <0.001        Chi-squared
                        1                                    2211 (54.4)    327 (79.0)                           
Hypertension            0                                0   2536 (62.4)    216 (52.2)  <0.001        Chi-squared
                        1                                    1531 (37.6)    198 (47.8)                           
Diabetes                0                                0   3536 (86.9)    314 (75.8)  <0.001        Chi-squared
                        1                                     531 (13.1)    100 (24.2)                           
smoking                 0                                0   3802 (93.5)    373 (90.1)   0.012        Chi-squared
                        1                                      265 (6.5)      41 (9.9)                           
coronary.artery.disease 0                                0   3572 (87.8)    337 (81.4)  <0.001        Chi-squared
                        1                                     495 (12.2)     77 (18.6)                           
previous.stroke         0                                0   3754 (92.3)    314 (75.8)  <0.001        Chi-squared
                        1                                      313 (7.7)    100 (24.2)                           
Hyperlipidemia          0                                0   2699 (66.4)    281 (67.9)   0.571        Chi-squared
                        1                                    1368 (33.6)    133 (32.1)                           
수축기혈압                                                    0  145.7 (22.0)  148.3 (23.6)   0.032  Two Sample T-test
이완기혈압                                                    0   88.1 (14.0)   87.7 (15.8)   0.584  Two Sample T-test
맥박                                                       0   74.6 (14.0)   75.2 (15.1)   0.425  Two Sample T-test
Revisit                 0                                0   4020 (98.8)    406 (98.1)   0.257        Chi-squared
                        1                                       47 (1.2)       8 (1.9)                           
[1] Warning, Hartigan's Dip Test reports possible multimodal distributions for: age, 맥박, 수축기혈압, 이완기혈압.
[2] Warning, Tukey test indicates far outliers in: 맥박, 이완기혈압.
[3] Warning, test for normality reports non-normal distributions for: age, 맥박, 수축기혈압, 이완기혈압.

In [193]:
tab2.to_csv(f'{savedir}/../results/dnn/tableone_withLAB{wl}_{today_datever}.csv')
tab12=pd.read_csv(f'{savedir}/../results/dnn/tableone_withLAB{wl}_{today_datever}.csv')

In [194]:
idx0=np.where((tab12.iloc[:,1]=='0.0')|(tab12.iloc[:,1]=='0'))[0]
v=tab12.loc[(idx0).tolist(),'Grouped by Central.dizziness'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Central.dizziness']=v
v=tab12.loc[(idx0).tolist(),'Grouped by Central.dizziness.3'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Central.dizziness.3']=v
v=tab12.loc[(idx0).tolist(),'Grouped by Central.dizziness.4'].copy()
v.index=v.index+1
tab12.loc[(idx0+1),'Grouped by Central.dizziness.4']=v
tab12=tab12[~tab12.index.isin(idx0)]
tab12['Unnamed: 1']=tab12['Unnamed: 1'].replace('1.0','')
tab12['Unnamed: 1']=tab12['Unnamed: 1'].replace('1','')

contidx=tab12['Grouped by Central.dizziness.4']=='Two Sample T-test'
for cn in ['Grouped by Central.dizziness.1','Grouped by Central.dizziness.2']:
    tab12.loc[contidx,cn]=tab12.loc[contidx,cn].replace('\(',u'\u00b1',regex=True)
    tab12.loc[contidx,cn]=tab12.loc[contidx,cn].replace('\)','',regex=True)
tab12=tab12.loc[(~(tab12['Unnamed: 0']=='variable')),]
nd=tab12.iloc[1,3:5].astype(int).sum()
tab12.to_excel(f'{savedir}/../results/tableone/tableone_target_N{nd}_{today_datever}.xlsx',index=False)

In [186]:
pv=tmp['Grouped by Central.dizziness.3'].apply(str2float)
sig_flag=(tmp['Unnamed: 0']!='의식상태')&((pv<0.05)|(tmp['Grouped by Central.dizziness.3']=='<0.001'))
tmp.loc[sig_flag,'Unnamed: 0'].to_csv('%ssigvar_withLAB%s_%s.csv'%(savedir,wl,today_datever),index=False, header=['var'])

TypeError: string indices must be integers

In [ ]:
sum(sig_flag)

In [ ]:
pv.loc[sig_flag]

In [ ]:
tmp.loc[sig_flag,'Unnamed: 0']

In [ ]:
labvar=pd.Series(total_var)[~pd.Series(total_var).isin(feature_var)].to_list()

In [ ]:
TableOne(data=seldata.loc[seldata.Train==True,:], columns=labvar, categorical=[], groupby='Central.dizziness', pval=True, isnull=True)

In [ ]:
seldata.head()

In [ ]:
sum(seldata.loc[seldata.Train==True,'ID'].isin(seldata.loc[seldata.Train==False,'ID']))

In [ ]:
seldata.loc[:,seldata.columns.isin(['registry_no']+mrs_var)].to_csv('/home/hdd1/Projects/mRSprediction/data/mRSseldata.csv' ,index=False)

In [ ]:
seldata.loc[:,seldata.columns.isin(['registry_no']+mrs_var)].head()

### Revisited patients

In [ ]:
tmp=dizdata.ID.value_counts()
revisitID=tmp[tmp>1].index

In [ ]:
len(revisitID)

In [ ]:
tmp.loc[tmp['Revisit']==1,:]['Central.dizziness'].value_counts()/sum(tmp['Revisit']==1)

In [ ]:
dizdata.loc[dizdata['Revisit']==0,:]['Central.dizziness'].value_counts()/sum(dizdata['Revisit']==0)

In [ ]:
dizdata2=dizdata.copy()
dizdata2['Testset']=dizdata2.arrival_date>'2012-10-03 12:28:00'
trainID=np.unique(dizdata2.loc[~dizdata2['Testset'],:]['ID'])
tmp2=dizdata2.loc[(~dizdata2.ID.isin(trainID))|(~dizdata2.Testset),:] #excluded patients in Testset, which visited before 2012-10-13 12:28:00

In [ ]:
len(np.unique(dizdata.loc[dizdata.Revisit==1]['ID']))

In [ ]:
pd.crosstab(dizdata2.Testset,dizdata2.ID.isin(trainID))

In [ ]:
print(len(dizdata2))

In [ ]:
print(len(tmp2))
print(tmp2.Revisit.value_counts())
print(tmp2.Testset.value_counts())

In [ ]:
tmp=dizdata.loc[dizdata.ID.isin(revisitID),:]
tmp['Testset']=tmp.arrival_date>'2012-10-03 12:28:00'
tmp.sort_values(['ID', 'arrival_date'], ascending=[1, 1])

In [ ]:
tmp['row']=1

In [ ]:
print(sum(tmp.Testset==1))
print(sum(tmp.Testset==0))

In [ ]:
tmp[['ID','Testset','row','Central.dizziness']].groupby(by=['ID','Testset']).sum()

Notes

In [ ]:
import pandas as pd 
import numpy as np 

In [ ]:
tmp = pd.read_csv('/home/hdd1/Projects/dizziness/data/DizzinessDataset_withTestsetLabel_191127.csv')

In [ ]:
tmp.head()

In [ ]:
tmp.loc[tmp.Testset==1,:].ID.shape

In [ ]:
tmp.loc[tmp.Testset==1,:].ID.unique().shape

In [ ]:
tmp.loc[tmp.Testset==0,:].ID.shape

In [ ]:
tmp.loc[tmp.Testset==0,:].ID.unique().shape

In [ ]:
tmp.loc[tmp.Testset==0,:].ID.value_counts().value_counts()

In [ ]:
tmp.loc[tmp.Testset==1,:].ID.value_counts().value_counts()

In [ ]:
ts=tmp.loc[tmp.Testset==1,:]
tsrm=ts.loc[~ts.ID.isin(intersectID)]

In [ ]:
tsrm.ID.unique().shape

In [ ]:
tsrm.ID.value_counts().value_counts()

In [ ]:
np.where(tmp.loc[tmp.Testset==0,:].ID.isin(tmp.loc[tmp.Testset==1,:].ID))

In [ ]:
intersectindex

In [ ]:
intersectindex=np.where(tmp.loc[tmp.Testset==0,:].ID.isin(tmp.loc[tmp.Testset==1,:].ID))[0]
intersectID=tmp.loc[intersectindex,:].ID

In [ ]:
intersectDF=tmp.loc[tmp.ID.isin(intersectID)]

In [ ]:
intersectDF.sort_values(['ID','arrival_date']).to_csv('/home/hdd1/Projects/dizziness/data/CD_revisitpat_200316.csv')